In [114]:
from haystack.nodes import FARMReader
from haystack.document_stores import FAISSDocumentStore, ElasticsearchDocumentStore
from haystack.pipelines import ExtractiveQAPipeline
from pymongo import MongoClient
from haystack.schema import Document
from haystack.utils import print_answers
from haystack.nodes import BM25Retriever, PreProcessor
import pandas as pd
from sentence_transformers import SentenceTransformer, util, CrossEncoder
import numpy as np

from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

In [4]:
db_client = MongoClient(host="localhost", port=27017)
database = db_client["Website_Chatbot"]
collection = database["MitsSpider"]

In [91]:
docs = [] 

text_processor = PreProcessor(
    split_by="sentence",
    split_length=3,
    split_respect_sentence_boundary=False,
    # split_overlap=25
)

for document in collection.find({}):
    content = document["content"]
    for para in content.split("\n\n"):
        if not para.strip():
            continue
        


        tmp_doc = Document(content=para)
        tmp_doc.id = str(document["_id"])
        tmp_doc.meta = {"source":document['source'], }
        tmp_doc.content_type = "str"

        docs += (text_processor.process(tmp_doc))



C:\Users\ANT-PC\AppData\Local\Temp\ipykernel_13820\1427879221.py:23: DeprecationWarning: Using a single Document as argument to the 'documents' parameter is deprecated. Use a list of (a single) Document instead.
  docs += (text_processor.process(tmp_doc))


In [93]:
len(docs)
# docs[0]

710

In [94]:
content_list = [doc.content for doc in docs]
source_list = [doc.meta['source'] for doc in docs]

doc_df = pd.DataFrame({'content':content_list, 'source':source_list}, index=None)

In [57]:
model_name = "deepset/roberta-base-squad2"
reader_model = AutoModelForQuestionAnswering.from_pretrained(model_name)
reader_tokenizer = AutoTokenizer.from_pretrained(model_name)
nlp = pipeline("question-answering", model=reader_model, tokenizer=reader_tokenizer)

In [135]:
retriever = SentenceTransformer('msmarco-distilbert-base-tas-b')
# ranker = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

In [136]:
passage_emedding = retriever.encode(content_list)

In [145]:
query = "who is the head of the department of  "
query_embedding = retriever.encode(query)
similarity_score = util.semantic_search(query_embedding, passage_emedding, top_k=5)
# sorted_list = np.argsort(similarity_score)
# top_5 = sorted_list[0][-5:]
cross_inp = [ content_list[hit['corpus_id']] for hit in similarity_score[0]]
for doc in cross_inp:
    qa_input = {"question": query, "context": doc}
    print(nlp(qa_input))

{'score': 0.10398130863904953, 'start': 81, 'end': 93, 'answer': 'DrPriya C. V'}
{'score': 0.29943349957466125, 'start': 83, 'end': 102, 'answer': 'Girish Balakrishnan'}
{'score': 0.07784692943096161, 'start': 338, 'end': 356, 'answer': 'Ms.Dhanya Sudarsan'}
{'score': 6.872298399684951e-05, 'start': 211, 'end': 229, 'answer': 'Dr. Mary Lissy P.N'}
{'score': 0.4464295208454132, 'start': 98, 'end': 109, 'answer': 'Dr Raju C.K'}
